In [ ]:
import pandas as pd 


In [1]:
# Convert money lines to probabilites 
def to_prob(money_line):
    if money_line < 0: 
        prob = -money_line / (-money_line + 100) 
    elif money_line > 0: 
        prob = 100 / (money_line + 100) 
    else: 
        prob = None 

    return (prob) 

In [12]:
import requests
from bs4 import BeautifulSoup 
import pandas as pd 

url = 'https://projects.fivethirtyeight.com/2019-mlb-predictions/games/'
page_response = requests.get(url)
page_content = BeautifulSoup(page_response.text, 'html.parser')

In [44]:
rows = page_content.findAll(class_="tr")

dict_list = []

for row in rows:
    try:
        date = row.find(class_='td td-datetime').getText()
    except:
        score = None
    name = row.find(class_='team-name long').getText()
    num_rating = row.find(class_='td number td-number rating-adj').getText().replace('=', '')
    win_prob = row.find(class_='td number td-number win-prob').getText()
    
    try:
        score = row.find(class_='td number td-number score').getText()
    except:
        score = None
    
    d = {
        'date': date,
        'name': name,
        'num_rating': num_rating, 
        'win_prob': win_prob,
        'score': score
    }
    
    dict_list.append(d)
    
pd.DataFrame(dict_list)

,date,name,num_rating,score,win_prob
0,"7/15Monday, July 157 p.m. Eastern",Dodgers,1584,,58%
1,"7/15Monday, July 157 p.m. Eastern",Phillies,1531,,42%
2,"7/15Monday, July 157:05 p.m.",Rays,1546,,41%
3,"7/15Monday, July 157:05 p.m.",Yankees,1610,,59%
4,"7/15Monday, July 157:10 p.m.",Blue Jays,1467,,38%
5,"7/15Monday, July 157:10 p.m.",Red Sox,1555,,62%
6,"7/15Monday, July 157:10 p.m.",Tigers,1414,,34%
7,"7/15Monday, July 157:10 p.m.",Indians,1525,,66%
8,"7/15Monday, July 158:05 p.m.",Reds,1530,,44%
9,"7/15Monday, July 158:05 p.m.",Cubs,1574,,56%
